In [1]:
import pandas as pd
import json

animes = pd.read_csv("anime_cleaned.csv", usecols = ["anime_id"])

#with open('voice_actors.json', 'r') as f:
#    voiceActors = json.load(f)

In [12]:
from jikanpy import Jikan
import time

jikan = Jikan()

head = animes.head(2)
voiceActors = dict()
fails = []

for i in animes.iloc[:,0]:
    #try :
        characters = jikan.anime(1000, extension='characters_staff').get('characters')
        voiceActors[i] = dict()
        for c in characters:
            actuals = c.get('voice_actors')
            for v in actuals:
                language = v.get('language')
                if not language in voiceActors[i]:
                    voiceActors[i][language] = list()
                voiceActors[i][language].append(v.get('mal_id'))
    #except :
        fails.append(i)
        print(i)
        time.sleep(2)

ConnectionError: HTTPSConnectionPool(host='api.jikan.moe', port=443): Max retries exceeded with url: /v3/anime/1000/characters_staff (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000000008FBBB00>: Failed to establish a new connection: [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu',))

In [11]:
import json

with open('voice_actors_infos.json', 'r') as infile:
    voiceActorInfos = json.load(infile)
with open('vA_allActors.json', 'r') as infile:
    allActors = json.load(infile)

In [ ]:
import json

with open('voice_actors.json', 'r') as outfile:
    json.dump(voiceActors, outfile)

In [236]:
pprint(fails)
#pprint(jikan.anime(fails[1], extension='characters_staff'))

[36519,
 37032,
 31930,
 34848,
 37005,
 10945,
 16287,
 32089,
 12723,
 36327,
 10520,
 37019]


In [4]:
voiceActorsInfos = dict()
fails_chars = []
count = 0

for lang, actors in allActors.items():
    for actor in actors:
        try :
            print(actor)
            v_a = jikan.person(actor)
            voiceActorsInfos[actor] = v_a 
        except :
            fails_chars.append(actor)
        time.sleep(2)
        if(count % 100 == 0):
            print(count)
        count+=1

NameError: name 'allActors' is not defined

In [7]:
allActors = dict()

for anime in voiceActors.items():
    #pprint(anime[1])
        for k, v in anime[1].items():
            if not k in allActors:
                allActors[k] = set()
            for actor in v:
                if actor != "":
                    allActors[k].add(actor)

In [107]:
voiceActors = dict()

for lang, actors in animesPerActor.items():
    for actor, animes in actors.items():
        for anime in animes:
            if not anime in voiceActors:
                voiceActors[anime] = dict()
            if not lang in voiceActors[anime]:
                voiceActors[anime][lang] = set()
            
            voiceActors[anime][lang].add(actor)

In [13]:
def findLanguage(actor):
    for language, actors in allActors.items():
        for a in actors:
            if (int(a) == int(actor)):
                return language

In [21]:
animesPerActor = dict()

for id, actor in voiceActorInfos.items():
    lang = findLanguage(id)
    if not lang in animesPerActor:
        animesPerActor[lang] = dict()
    animesPerActor[lang][id] = set()
    for anime in actor['voice_acting_roles']:
        animesPerActor[lang][id].add(anime['anime']['mal_id'])

In [79]:
def topFromLanguage(actors, top):
    actorsFromLanguage = dict(filter(lambda elem: int(elem[0]) in actors, voiceActorInfos.items()))
    sort = sorted(actorsFromLanguage.items(), reverse=True, key=lambda x: len(x[1]['voice_acting_roles']))[:top]
    return sort

In [109]:
def groupsByLang():
    groupsByLang = dict()
    
    for top in [5, 10, 20, 30, 40]:
        actorsPerLanguage = dict()
        groupsByLang[top] = dict()
        for lang, actors in allActors.items():
            actorsPerLanguage[lang] = topFromLanguage(actors, top)
        
        print("top "+str(top))
        for lang, actors in actorsPerLanguage.items():
            groupsByLang[top][lang] = actorsByLanguage(actors)
    
    return groupsByLang

In [110]:
def actorsByLanguage(sampleFrom):
    sample = []
    for actor in sampleFrom:
        totalActors.add(int(actor[0]))
        sample.append(int(actor[0]))
        
    langArray = dict()
    for anime in voiceActors.items():
        curr = anime[0]
        for k, v in anime[1].items():
            for actor in v:
                if int(actor) in sample:
                    if not curr in langArray:
                        langArray[curr] = set()
                    langArray[curr].add(actor)
        if curr in langArray:
            langArray[curr] = list(langArray[curr])
    return langArray

In [116]:
def isLinkPresent(data, x, y):
    if {"from": x, "to": y, "value": 1, "id":0} in data or {"from": y, "to": x, "value": 1, "id":0} in data:
        return True
    else:
        return False

In [83]:
def colorByGender(gender):
    if gender == "M": 
        return "#93B5C6"
    elif gender == "F": 
        return "#BD4F6C"
    else:
        return "#111111"

In [84]:
def colorByPopu(popu):
    color = "white"
    if (popu < 250):
        color = "green"
    elif (popu < 1000):
        color = "yellow"
    elif (popu < 2500):
        color = "orange"
    else:
        color = "red"
    return color

In [85]:
def dataOfActors(actors):
    data = []
    for actor in actors:
        if str(actor) in voiceActorInfos:
            infos = voiceActorInfos.get(str(actor))
            data.append({
                "from" : infos.get('name'), 
                "name" : infos.get('name'),
                "id" : infos.get('mal_id'),
                "colorByGender": colorByGender("M"), 
                "popularity": infos.get('member_favorites'),
                "colorByPopu": colorByPopu(infos.get('member_favorites')), 
                "number": len(infos.get('voice_acting_roles')),
            })
    return data

In [117]:
dataAll = dict()
totalActors = set()

for top, items in groupsByLang().items():
    dataAll[top] = dict()
    for lang, rest in items.items():
        dataAll[top][lang] = []
        for anime, actors in rest.items():
            for item in dataOfActors(actors):
                if not item in dataAll[top][lang]:
                    dataAll[top][lang].append(item)
            pairs = [(actors[p1], actors[p2]) for p1 in range(len(actors)) for p2 in range(p1+1,len(actors))]
            for x,y in pairs:
                x = voiceActorInfos.get(str(x)).get('name')
                y = voiceActorInfos.get(str(y)).get('name')
                if not isLinkPresent(dataAll[top][lang], x, y):
                    dataAll[top][lang].append({"from": x, "to": y, "value": 1, "id": 0})

top 5
top 10
top 20
top 30
top 40


In [100]:
actualVAInfos = dict()
csv = pd.read_csv("anime_cleaned.csv", usecols = ["anime_id", "favorites"])
csv.set_index("anime_id", inplace=True, drop=True)

def checkElem(elem):
    if elem["anime"]["mal_id"] in csv.index:
        return int(csv.loc[elem["anime"]["mal_id"]]["favorites"])
    else: return 0

def processAnimes(animes):
    animes = sorted(animes, reverse=True, key=checkElem)[:3]
    ret = []
    count = 0
    for rest in animes:
        ret.append(dict())
        ret[count]["anime"] = rest["anime"]["name"]
        ret[count]["character"] = rest["character"]["name"]
        ret[count]["image"] = rest["character"]["image_url"]
        count += 1
    
    return ret

def getKeys(vaInfos):
    infos = dict()
    infos["id"] = vaInfos["mal_id"]
    infos["name"] = vaInfos["name"]
    infos["birthday"] = vaInfos["birthday"]
    infos["about"] = vaInfos["about"]
    infos["image_url"] = vaInfos["image_url"]
    infos["animes"] = processAnimes(vaInfos["voice_acting_roles"])
    
    return infos

for actor in totalActors:
    actualVAInfos[str(actor)] = getKeys(voiceActorInfos[str(actor)])

In [102]:
import json

with open('C:/Users/Joachim/Documents/Git/repository/com-480-project-worldwideweebz/app/src/pages/chord/vA_infos.json', 'w') as outfile:
    json.dump(actualVAInfos, outfile)

In [118]:
import json

with open('C:/Users/Joachim/Documents/Git/repository/com-480-project-worldwideweebz/app/public/data/vA_datasets.json', 'w') as outfile:
    json.dump(dataAll, outfile)

In [112]:
print(dataAll)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

